# Inroduction

Unlike the other database in the pipeline the oracle database can't be automatically configured for a cdc pipeline. There are alot of changes that needs to be done to initial databse that was intialized when the continer comes up.

For instance while the other containerized database takes under  minutes to build the oracle containeried database takes close to 40 minute to initiate the databse from the docker image. this only happens the firsttime a container is being built from the images the time need for initializtion decreases subsequently.

In [2]:
# Import libraries
import pymssql
import random
import uuid
from decimal import Decimal
import datetime
import time

# step 1 

Run the log-miner script provided by oracle this script setup the logminer on the databse table which we would like to capture data changes from.

- Connect to the oracle container using a shell section
- run the following command 
```sh
curl https://raw.githubusercontent.com/debezium/oracle-vagrant-box/master/setup-logminer.sh | sh
```
essentially what this code does is to get the logminer.sh script from the oracle repo and pipe it to the running session. Logminer is setup, New users are created and granted required permission.

# Step 2 Increase recovery area size

The recovery area data allocated to the logminer is defaulted to 10gb we need to increase this to 20gb to have more available space. Note the recovery area size can be increased to according to the needs. The cell below has been configureed to increase the recovery area size to 20G you can easily switch out the 20G in the query to your desired size.

In [6]:
import oracledb

conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLCDB",
    user="system",
    password="top_secret"
)

# Create a cursor
cursor = conn.cursor()

# Define your query to create the table
query = '''
  alter system set db_recovery_file_dest_size = 20G
'''

# Execute the query
cursor.execute(query)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

# Step 3 create table

The next step is to create the cusreads table which our debezium connector is going to capture changes. The `debezium` user which was created by the logminer.sh script is used to create this table. The cell below as been written to create the required table in this case CUSREADS.




In [9]:
import oracledb

conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLPDB1",
    user="debezium",
    password="dbz"
)

# Create a cursor
cursor = conn.cursor()

# Define your query to create the table
query = '''
  CREATE TABLE CUSREADS (
        rowno RAW(16) PRIMARY KEY,
        cust NUMBER(10,0),
        loc NUMBER(10,0),
        meter VARCHAR2(32),
        consumption NUMBER(7,2),
        consumdt TIMESTAMP,
        rdrtype CHAR(1)
    )
'''

# Execute the query
cursor.execute(query)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

# Step 4 grant select permission to user 

The `logminer-setup.sh` created two users the `debezium` and `c##dbzuser`. User `debezium` is used to create new tables while user `c##dbzuser` is used to by the debezium connector to connect to the oracle service `ORCLPDB1`. The user `debezium` needs to grant select permission to `c##dbzuser` in order for logminer to capture changes.

#### NOTE: 
Over the course of this notebook we have used two different service name `ORCLCDB` and `ORCLPDB1`. The first is the main containerized database which can be accessed using teh system user and password while the later is the pluggable service name for the oracle db this ervice can be accessed by the the created users. you can read up on the oracle CDB for context on the architecture.

In [10]:

conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLPDB1",
    user="debezium",
    password="dbz"
)

# Create a cursor
cursor = conn.cursor()

# Define your query to create the table
query = '''
  GRANT SELECT ON CUSREADS to c##dbzuser
'''

# Execute the query
cursor.execute(query)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

# Step 5 alter table cusread to stream all changes 

With the current configuration the cdc messages that will be produced by kafka will be minimal and won't be enough for the pipeline we need kafka to strea more details such as operation and the full table. The cell below will alter the ` cusreads` table to allow kafka read all columns in it's message.

In [11]:


conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLPDB1",
    user="debezium",
    password="dbz"
)

# Create a cursor
cursor = conn.cursor()

# Define your query to create the table
query = '''
  ALTER TABLE DEBEZIUM.CUSREADS ADD SUPPLEMENTAL LOG DATA (ALL) COLUMNS
'''

# Execute the query
cursor.execute(query)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

# select from table cusreads

In [12]:
import oracledb

conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLPDB1",
    user="debezium",
    password="dbz"
)


cursor = conn.cursor()

query = ''' 
        SELECT * FROM CUSREADS
        '''

cursor.execute(query)
results = cursor.fetchall()

for row in results[:]:
    print(row)

# insert to table

In [3]:
import oracledb
from datetime import datetime

conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLPDB1",
    user="debezium",
    password="dbz"
)


cursor = conn.cursor()

# Function to insert random data
def insert_random_row(cursor):
    rowno = uuid.uuid4().bytes
    cust = random.randint(1, 10000)
    loc = random.randint(1, 10000)
    meter = "METER" + str(random.randint(1, 100))
    consumption = round(random.uniform(0, 100), 2)
    consumdt = datetime.now()
    rdrtype = random.choice(['I']) 

    insert_sql = """
        INSERT INTO CUSREADS (rowno, cust, loc, meter, consumption, consumdt, rdrtype)
        VALUES (:rowno, :cust, :loc, :meter, :consumption, :consumdt, :rdrtype)
    """
    cursor.execute(insert_sql, rowno=rowno, cust=cust, loc=loc, meter=meter, consumption=consumption, consumdt=consumdt, rdrtype=rdrtype)

# Insert N random rows
N = 5 # for example, to insert 10 random rows
for _ in range(N):
    insert_random_row(cursor)

conn.commit()
cursor.close()
conn.close()

# Step 6 increase redo size
So far we've been able to create a table, write data to the table and select rows from the created tables.

One more thing we nweed to do before we can intiate our debezium connector is to increse the size of the redo logs. Only 200mb is assigned to each log group by default so we need to increase this to around 500mb. The only way we ae able to achieve this is to delete and recreate the group one after the other.

- To do this we need to know which group is active, current or unused. By running the the immediate vcell below we can see the status of each group. 
- Then modify the cell named Alter just 2cell below by changing the group number to the unused number and also the the x in `/opt/oracle/oradata/ORCLCDB/redo0x.log` for instance if the unused group is 2 the code should look like this 
```python
# Alter
import oracledb

conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLCDB",
    user="system",
    password="top_secret"
)


# Create a cursor
cursor = conn.cursor()

# Execute the statements one by one
cursor.execute("ALTER DATABASE CLEAR LOGFILE GROUP 2")
cursor.execute("ALTER DATABASE DROP LOGFILE GROUP 2")
cursor.execute("ALTER DATABASE ADD LOGFILE GROUP 2 ('/opt/oracle/oradata/ORCLCDB/redo02.log') size 500M REUSE")


#cursor.execute("ALTER SYSTEM SWITCH LOGFILE")

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()
```


- once this is done you can rerun the select cell the size for the group we choose should have changed. do the same for all unsused group.
- lastly for the group tagged current you need to switch the current log file by running the 3rd cell below named switch. the change won't take effect immediately so you have to wait up to 10 min for the status of the current group to change to unused once this has changed you can modify cell 2 to increase the log size.

In [26]:
# select 

import oracledb

conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLPDB1",
    user="system",
    password="top_secret"
)


cursor = conn.cursor()

query = ''' 
        SELECT GROUP#, BYTES/1024/1024 SIZE_MB, STATUS FROM V$LOG ORDER BY 1
        '''

cursor.execute(query)
results = cursor.fetchall()

for row in results[:]:
    print(row)

(1, 500, 'UNUSED')
(2, 500, 'CURRENT')
(3, 500, 'UNUSED')


In [25]:
# Alter
import oracledb

conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLCDB",
    user="system",
    password="top_secret"
)


# Create a cursor
cursor = conn.cursor()

# Execute the statements one by one
cursor.execute("ALTER DATABASE CLEAR LOGFILE GROUP 1")
cursor.execute("ALTER DATABASE DROP LOGFILE GROUP 1")
cursor.execute("ALTER DATABASE ADD LOGFILE GROUP 1 ('/opt/oracle/oradata/ORCLCDB/redo01.log') size 500M REUSE")


#cursor.execute("ALTER SYSTEM SWITCH LOGFILE")

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()


# switch log file and wait 5 to 10 min for group1 to changr to unused


In [19]:
import oracledb

conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLCDB",
    user="system",
    password="top_secret"
)


# Create a cursor
cursor = conn.cursor()



cursor.execute("ALTER SYSTEM SWITCH LOGFILE")

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()


# delete table

In [4]:
conn = oracledb.connect(
    host="192.168.98.31",
    port="1521",
    service_name="ORCLPDB1",
    user="debezium",
    password="dbz"
)



# Create a cursor
cursor = conn.cursor()

# Execute the statements one by one
cursor.execute("DELETE FROM CUSREADS WHERE CUST = 4172")


# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()



# step 7 starting the pipeline

we've completed the configuration. 
- To start the pipeline to database we need to attach a shell to the python app container and run the following code. 

`python3 oracle_pipeline/src/debezium_to_database_oracle.py`

- To start the pipeline to api endpoint we need to attach another shell to the python app container and run the following code. 

`python3 oracle_pipeline/src/debezium_to_oracle_api.py`


The pipelines are up it should take few minute for logminer to start reading from oracle table and stream the changes to kafka.